In [1]:
import os
import cv2
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
import random
import time

https://drive.google.com/drive/folders/1_MU7jTR0kTt8MFQYrUzWiWc6uM-BHG_9?usp=drive_link

In [2]:
def load_data(dataset_path):
    data = []
    labels = []

    for folder in os.listdir(dataset_path):
        folder_path = os.path.join(dataset_path, folder)
        if os.path.isdir(folder_path):
            label = 1 if folder.lower() == 'ok' else 0  # 1 for OK, 0 for defects
            for filename in os.listdir(folder_path):
                img_path = os.path.join(folder_path, filename)
                img = cv2.imread(img_path)
                img = cv2.resize(img, (128, 128))  # Adjust the size as needed
                data.append(img)
                labels.append(label)
                
    data = np.array(data) / 255.0  # Normalize pixel values
    labels = np.array(labels)

    return data, labels

In [3]:
dataset_path = 'dataset'
data, labels = load_data(dataset_path)


In [4]:
print(len(data))

324


In [5]:
model = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])



In [7]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(data, labels, epochs=10, validation_split=0.20)

#model.save('defect_detection_model.h5')

Epoch 1/10
9/9 [==============================] - 4s 295ms/step - loss: 0.4042 - accuracy: 0.8108 - val_loss: 1.0875 - val_accuracy: 0.4615
Epoch 2/10
9/9 [==============================] - 3s 278ms/step - loss: 0.3580 - accuracy: 0.8571 - val_loss: 0.4670 - val_accuracy: 0.7846
Epoch 3/10
9/9 [==============================] - 2s 252ms/step - loss: 0.2725 - accuracy: 0.8996 - val_loss: 0.3408 - val_accuracy: 0.8000
Epoch 4/10
9/9 [==============================] - 2s 247ms/step - loss: 0.3005 - accuracy: 0.8958 - val_loss: 0.5987 - val_accuracy: 0.7385
Epoch 5/10
9/9 [==============================] - 2s 238ms/step - loss: 0.2109 - accuracy: 0.9305 - val_loss: 0.6808 - val_accuracy: 0.7231
Epoch 6/10
9/9 [==============================] - 2s 245ms/step - loss: 0.1723 - accuracy: 0.9382 - val_loss: 0.8069 - val_accuracy: 0.6769
Epoch 7/10
9/9 [==============================] - 2s 250ms/step - loss: 0.1443 - accuracy: 0.9537 - val_loss: 0.4313 - val_accuracy: 0.8000
Epoch 8/10
9/9 [====

In [8]:
def load_and_preprocess_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (128, 128))  # Assurez-vous que la taille correspond à celle utilisée lors de l'entraînement
    img = img / 255.0  # Normalisez les valeurs des pixels
    img = np.expand_dims(img, axis=0)  # Ajoutez une dimension pour représenter le lot (batch)

    return img

def predict_defect(model, image_path):
    preprocessed_img = load_and_preprocess_image(image_path)
    prediction = model.predict(preprocessed_img)

    # Interprétez la prédiction (1 signifie "OK", 0 signifie "défaut")
    if prediction[0, 0] > 0.5:
        return "OK"
    else:
        return "Défaut"



In [10]:
# Charger le modèle préalablement entraîné
#model = keras.models.load_model('defect_detection_model.h5')

# Exemple d'utilisation
image_path_to_predict = 'ok.jpg'
result = predict_defect(model, image_path_to_predict)
print(f"Prédiction pour {image_path_to_predict}: {result}")

error: OpenCV(4.8.1) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4062: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'


In [110]:
resultf=""
for filename in os.listdir("testperso"):
    #print(filename)
    result = predict_defect(model, "testperso/" + filename)
    resultf= resultf + "prédiction pour"+ filename + " : "+ str(result) +"\n"
    


1/1 [==============================] - 0s 16ms/step


In [111]:
print (resultf)

prédiction pourWIN_20231010_15_02_24_Pro.jpg : Défaut
prédiction pourWIN_20231010_15_02_40_Pro.jpg : Défaut
prédiction pourWIN_20231010_15_03_00_Pro.jpg : Défaut
prédiction pourWIN_20231010_15_03_09_Pro.jpg : Défaut
prédiction pourWIN_20231010_15_03_11_Pro.jpg : Défaut
prédiction pourWIN_20231010_15_03_18_Pro.jpg : Défaut
prédiction pourWIN_20231010_15_03_24_Pro.jpg : OK
prédiction pourWIN_20231010_15_03_29_Pro.jpg : Défaut
prédiction pourWIN_20231010_15_03_56_Pro.jpg : Défaut
prédiction pourWIN_20231010_15_03_58_Pro.jpg : Défaut
prédiction pourWIN_20231010_15_04_02_Pro.jpg : Défaut
prédiction pourWIN_20231010_15_04_05_Pro.jpg : Défaut



multi classes-----------------------------

In [11]:
def load_and_preprocess_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (128, 128))  # Assurez-vous que la taille correspond à celle utilisée lors de l'entraînement
    img = img / 255.0  # Normalisez les valeurs des pixels
    img = np.expand_dims(img, axis=0)  # Ajoutez une dimension pour représenter le lot (batch)

    return img

def predict_defect(model, image_path):
    preprocessed_img = load_and_preprocess_image(image_path)
    prediction = model.predict(preprocessed_img)

    # Interprétez la prédiction (1 signifie "OK", 0 signifie "défaut")
    if prediction[0, 0] > 0.5:
        return "OK"
    else:
        return "Défaut"


In [12]:
def load_data_multi_class(dataset_path):
    data = []
    labels = []

    for folder in os.listdir(dataset_path):
        folder_path = os.path.join(dataset_path, folder)
        if os.path.isdir(folder_path):
            
            label = folder.lower()  # Utilisez le nom du dossier comme label
            print(label)
            for filename in os.listdir(folder_path):
                img_path = os.path.join(folder_path, filename)
                img = cv2.imread(img_path)
                img = cv2.resize(img, (128, 128))
                data.append(img)
                labels.append(label)

    data = np.array(data) / 255.0
    labels = np.array(labels)

    return data, labels

In [13]:
dataset_path = 'dataset'
data, labels = load_data_multi_class(dataset_path)

blobs
cracks
ok
spaghetti
stringing
under exstrosion


In [14]:
# Définir un dictionnaire de mapping catégorie -> numéro
categorie_mapping = {
    "blobs": 0,
    "cracks": 1,
    "ok": 2,
    "spaghetti": 3,
    "stringing": 4,
    "under exstrosion": 5
}

# Utiliser NumPy pour convertir les catégories en numéros
labels = np.array([categorie_mapping[c] for c in labels])

In [15]:
print (len(labels))
print(labels[-1])

324
5


In [16]:
combined_data = list(zip(data,labels))

# Mélangez la liste combinée
random.shuffle(combined_data)

# Divisez à nouveau la liste combinée en deux listes distinctes
data,labels = zip(*combined_data)

labels = np.array(labels)
data = np.array(data)
print (len(labels))
print(labels[-1])

324
5


In [37]:
model = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dropout(0.5),  # Ajout de la couche Dropout
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),  # Ajout d'une autre couche Dropout
    layers.Dense(10, activation='softmax')  # Ajustez cela en fonction de votre nombre de classes
])

In [41]:
data.shape

259.2


In [42]:

# Compilez le modèle
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Entraînez le modèle

model.fit(data, labels, epochs=10, validation_split=0.20, batch_size=10)


Epoch 1/10
26/26 [==============================] - 5s 163ms/step - loss: 0.2009 - accuracy: 0.9575 - val_loss: 2.6285 - val_accuracy: 0.3538
Epoch 2/10
26/26 [==============================] - 4s 144ms/step - loss: 0.1539 - accuracy: 0.9498 - val_loss: 2.3858 - val_accuracy: 0.3692
Epoch 3/10
26/26 [==============================] - 4s 149ms/step - loss: 0.1376 - accuracy: 0.9653 - val_loss: 3.3681 - val_accuracy: 0.3846
Epoch 4/10
26/26 [==============================] - 4s 155ms/step - loss: 0.1189 - accuracy: 0.9691 - val_loss: 2.9669 - val_accuracy: 0.3231
Epoch 5/10
26/26 [==============================] - 4s 153ms/step - loss: 0.1050 - accuracy: 0.9730 - val_loss: 3.7586 - val_accuracy: 0.3231
Epoch 6/10
26/26 [==============================] - 4s 151ms/step - loss: 0.0884 - accuracy: 0.9768 - val_loss: 3.9973 - val_accuracy: 0.4000
Epoch 7/10
26/26 [==============================] - 4s 155ms/step - loss: 0.0648 - accuracy: 0.9846 - val_loss: 3.6161 - val_accuracy: 0.4000
Epoch 

In [32]:
def predict_defect_multi_class(model, image_path):
    preprocessed_img = load_and_preprocess_image(image_path)
    predictions = model.predict(preprocessed_img)
    print (model.predict(preprocessed_img))

    # Interprétez les prédictions
    class_labels = ['blobs', 'cracks', 'OK', 'spaghetti','under exstrosion']
    predicted_class = class_labels[np.argmax(predictions)]

    return predicted_class

In [33]:
resultf=""
nb_total=0
nb_positif=0
for filename in os.listdir("testperso"):
    #print(filename)
    nb_total+=1

    result = predict_defect_multi_class(model, "testperso/" + filename)
    if result=='OK':
        nb_positif+=1
        
    resultf= resultf + "prédiction pour"+ filename + " : "+ str(result) +"\n"


1/1 [==============================] - 0s 25ms/step
[[3.5722216e-04 2.1220310e-05 9.8947161e-01 1.9746816e-03 6.8374928e-03
  1.3376599e-03 1.5739394e-09 2.9612368e-08 3.3252301e-09 8.2533829e-09]]
1/1 [==============================] - 0s 24ms/step
[[3.9721108e-06 2.3858815e-09 9.9990046e-01 8.9471810e-05 5.7368297e-06
  3.6077688e-07 6.5878669e-17 5.7095916e-14 5.0846150e-17 1.7270075e-15]]
1/1 [==============================] - 0s 30ms/step
[[2.2435676e-04 1.8971813e-08 9.9709177e-01 1.6477639e-03 1.0332337e-03
  2.9194962e-06 4.8622440e-13 2.3878970e-11 2.0624945e-13 4.5924419e-12]]
1/1 [==============================] - 0s 37ms/step
[[7.5437260e-05 6.4280332e-09 1.0377671e-02 1.7039272e-07 9.8954672e-01
  2.6728662e-08 9.2237606e-17 1.1863405e-14 5.3128477e-17 1.7707797e-15]]
1/1 [==============================] - 0s 25ms/step
[[5.7595023e-03 3.6498182e-10 9.3074250e-01 6.3492529e-02 3.7928353e-07
  5.0542021e-06 1.4904836e-18 3.8932536e-17 1.7266045e-19 1.1036398e-17]]
1/1 [=====

In [34]:
print (resultf)
print(str((nb_positif*100)/nb_total) + " % de précision")

prédiction pourok.jpg : OK
prédiction pourok3.jpg : OK
prédiction pourok4.jpg : OK
prédiction pourok5.png : under exstrosion
prédiction pourspaghetti.jpg : OK
prédiction pourspaghetti2.jpeg : OK
prédiction pourWIN_20231010_15_02_24_Pro.jpg : under exstrosion
prédiction pourWIN_20231010_15_02_40_Pro.jpg : spaghetti
prédiction pourWIN_20231010_15_03_00_Pro.jpg : spaghetti
prédiction pourWIN_20231010_15_03_09_Pro.jpg : OK
prédiction pourWIN_20231010_15_03_11_Pro.jpg : OK
prédiction pourWIN_20231010_15_03_18_Pro.jpg : OK
prédiction pourWIN_20231010_15_03_24_Pro.jpg : OK
prédiction pourWIN_20231010_15_03_29_Pro.jpg : OK
prédiction pourWIN_20231010_15_03_56_Pro.jpg : OK
prédiction pourWIN_20231010_15_03_58_Pro.jpg : OK
prédiction pourWIN_20231010_15_04_02_Pro.jpg : under exstrosion
prédiction pourWIN_20231010_15_04_05_Pro.jpg : OK

72.22222222222223 % de précision


In [35]:
# Ouvrir la webcam (la webcam par défaut a l'ID 0)
cap = cv2.VideoCapture(0)

# Vérifier si la webcam est ouverte correctement
if not cap.isOpened():
    print("Erreur: Impossible d'ouvrir la webcam.")
    exit()

# Capturer une image
time.sleep(1)
ret, frame = cap.read()

# Sauvegarder l'image capturée
if ret:
    cv2.imwrite("photo_capturee.jpg", frame)
    print("Photo capturée avec succès.")
else:
    print("Erreur lors de la capture de la photo.")

# Libérer la webcam
cap.release()
time.sleep(1)


result = predict_defect_multi_class(model, 'photo_capturee.jpg')
print(f"Prédiction pour {image_path_to_predict}: {result}")

Photo capturée avec succès.
1/1 [==============================] - 0s 32ms/step
[[3.4101017e-02 5.2544610e-03 9.1146988e-01 1.8071422e-02 1.5108110e-02
  1.5050286e-02 2.2792644e-04 2.9491232e-04 1.6262161e-04 2.5927747e-04]]
Prédiction pour ok.jpg: OK
